In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [2]:
# load data
df=pd.read_csv('PageBlocks_withoutdupl_02_v02.csv')  

del df['id']
del df['Unnamed: 0']

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,outlier
0,5.0,7.0,35.0,1.400,0.400,0.657,2.33,14.0,23.0,6.0,0
1,6.0,7.0,42.0,1.167,0.429,0.881,3.60,18.0,37.0,5.0,0
2,6.0,18.0,108.0,3.000,0.287,0.741,4.43,31.0,80.0,7.0,0
3,5.0,7.0,35.0,1.400,0.371,0.743,4.33,13.0,26.0,3.0,0
4,6.0,3.0,18.0,0.500,0.500,0.944,2.25,9.0,17.0,4.0,0


In [3]:
df.shape

(4982, 11)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/PageBlocks.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,10]
X = df[:,0:10]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
          # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 14:57:47,542][cascade_classifier.fit_transform] X_groups_train.shape=[(3337, 10)],y_train.shape=(3337,),X_groups_test.shape=[(1645, 10)],y_test.shape=(1645,)
[ 2018-04-26 14:57:47,544][cascade_classifier.fit_transform] group_dims=[10]
[ 2018-04-26 14:57:47,545][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:57:47,547][cascade_classifier.fit_transform] group_ends=[10]
[ 2018-04-26 14:57:47,548][cascade_classifier.fit_transform] X_train.shape=(3337, 10),X_test.shape=(1645, 10)
[ 2018-04-26 14:57:47,550][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3337, 10), X_cur_test.shape=(1645, 10)
[ 2018-04-26 14:57:48,371][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.51%
[ 2018-04-26 14:57:49,524][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.21%
[ 2018-04-26 14:57:50,460][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:57:58,243][cascade_classifier.transform] X_groups_test.shape=[(1645, 10)]
[ 2018-04-26 14:57:58,246][cascade_classifier.transform] group_dims=[10]
[ 2018-04-26 14:57:58,247][cascade_classifier.transform] X_test.shape=(1645, 10)
[ 2018-04-26 14:57:58,249][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1645, 10)


Test Accuracy of ExtraTrees = 98.662614 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 14:58:00,725][cascade_classifier.transform] X_groups_test.shape=[(1645, 10)]
[ 2018-04-26 14:58:00,727][cascade_classifier.transform] group_dims=[10]
[ 2018-04-26 14:58:00,729][cascade_classifier.transform] X_test.shape=(1645, 10)
[ 2018-04-26 14:58:00,731][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1645, 10)


In [11]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 14:58:03,230][cascade_classifier.fit_transform] X_groups_train.shape=[(3337, 12)],y_train.shape=(3337,),X_groups_test.shape=[(1645, 12)],y_test.shape=(1645,)
[ 2018-04-26 14:58:03,233][cascade_classifier.fit_transform] group_dims=[12]
[ 2018-04-26 14:58:03,235][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:58:03,236][cascade_classifier.fit_transform] group_ends=[12]
[ 2018-04-26 14:58:03,237][cascade_classifier.fit_transform] X_train.shape=(3337, 12),X_test.shape=(1645, 12)
[ 2018-04-26 14:58:03,240][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3337, 12), X_cur_test.shape=(1645, 12)
[ 2018-04-26 14:58:04,088][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.81%
[ 2018-04-26 14:58:05,160][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.21%
[ 2018-04-26 14:58:06,403][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:58:13,685][cascade_classifier.transform] X_groups_test.shape=[(1645, 12)]
[ 2018-04-26 14:58:13,686][cascade_classifier.transform] group_dims=[12]
[ 2018-04-26 14:58:13,687][cascade_classifier.transform] X_test.shape=(1645, 12)
[ 2018-04-26 14:58:13,690][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1645, 12)


Test Accuracy of ExtraTrees = 99.027356 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print (" Time ",format(round(tt,3))," seconds")

[ 2018-04-26 14:58:16,331][cascade_classifier.fit_transform] X_groups_train.shape=[(3337, 12)],y_train.shape=(3337,),X_groups_test.shape=[(1645, 12)],y_test.shape=(1645,)
[ 2018-04-26 14:58:16,332][cascade_classifier.fit_transform] group_dims=[12]
[ 2018-04-26 14:58:16,334][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:58:16,335][cascade_classifier.fit_transform] group_ends=[12]
[ 2018-04-26 14:58:16,336][cascade_classifier.fit_transform] X_train.shape=(3337, 12),X_test.shape=(1645, 12)
[ 2018-04-26 14:58:16,338][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3337, 12), X_cur_test.shape=(1645, 12)
[ 2018-04-26 14:58:17,185][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.51%
[ 2018-04-26 14:58:18,289][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.21%
[ 2018-04-26 14:58:19,358][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

('Test Accuracy of ExtraTrees = 98.905775 %', 0, 'fois')
(' Time ', '10.788', ' seconds')


[ 2018-04-26 14:58:30,429][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.51%
[ 2018-04-26 14:58:31,550][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.21%
[ 2018-04-26 14:58:32,633][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.21%
[ 2018-04-26 14:58:33,700][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.80%
[ 2018-04-26 14:58:34,784][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.40%
[ 2018-04-26 14:58:35,859][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.20%
[ 2018-04-26 14:58:36,998][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.80%
[ 2018-04-26 14:58:38,116][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.10%


('Test Accuracy of ExtraTrees = 98.784195 %', 1, 'fois')
(' Time ', '11.012', ' seconds')


[ 2018-04-26 14:58:44,094][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.81%
[ 2018-04-26 14:58:45,165][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.21%
[ 2018-04-26 14:58:46,246][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.81%
[ 2018-04-26 14:58:47,330][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.10%
[ 2018-04-26 14:58:48,404][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.40%
[ 2018-04-26 14:58:49,509][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.50%
[ 2018-04-26 14:58:50,634][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.80%
[ 2018-04-26 14:58:51,730][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.70%


...

('Test Accuracy of ExtraTrees = 98.784195 %', 17, 'fois')
(' Time ', '10.774', ' seconds')


[ 2018-04-26 15:02:17,211][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.21%
[ 2018-04-26 15:02:18,303][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.21%
[ 2018-04-26 15:02:19,363][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.51%
[ 2018-04-26 15:02:20,437][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.50%
[ 2018-04-26 15:02:21,535][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.70%
[ 2018-04-26 15:02:22,599][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.50%
[ 2018-04-26 15:02:23,693][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.80%
[ 2018-04-26 15:02:24,798][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.40%


('Test Accuracy of ExtraTrees = 98.844985 %', 18, 'fois')
(' Time ', '10.779', ' seconds')


[ 2018-04-26 15:02:30,374][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.51%
[ 2018-04-26 15:02:31,630][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.21%
[ 2018-04-26 15:02:32,714][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.21%
[ 2018-04-26 15:02:33,754][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.80%
[ 2018-04-26 15:02:34,824][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.40%
[ 2018-04-26 15:02:35,960][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.50%
[ 2018-04-26 15:02:36,946][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.80%
[ 2018-04-26 15:02:38,050][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.70%


('Test Accuracy of ExtraTrees = 98.844985 %', 19, 'fois')
(' Time ', '10.817', ' seconds')


# ('Test Accuracy of ExtraTrees = 99.148936 %', 1, 'fois')